In [35]:
import json
from uuid import uuid4
from collections import Counter

import pandas as pd
from tqdm import tqdm
import numpy as np
from datasets import load_dataset
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
import optuna
import shap
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import classification_report, f1_score

from imblearn.over_sampling import RandomOverSampler

from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

import pickle


# NER False Negatives

In [32]:
results_df = pd.read_csv('data/ner_deep_learning_results.csv')
mistakes = results_df[(results_df['y_pred'] == 0) & (results_df.model == 'dslim/bert-base-NER')]['Name'].to_numpy()
mistakes

array(['A', 'A', 'A', ..., 'Zion', 'Zion', 'Zion'], dtype=object)

# feature importance

In [40]:
model_name='dslim/bert-base-NER'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)
rng = np.random.default_rng(seed=0)
hard_name = rng.choice(mistakes)
print(hard_name)
switch_name = 'Peter'
targets = ['no emotion', 'anger', 'disgust', 'fear', 'happiness', 'sadness', 'surprise']


def train(who='all', n_samples=100, fp='data/output.pkl'):
    """
    who: 'all' - anonyimise everyone
    who: 'rare-once' - only insert the rare name once
    """

    removed_names = set()

    def anonymise(sentence, who='all'):
        global rare_insert_count
        nlp = pipeline("ner", model=model, tokenizer=tokenizer)
        ner_results = nlp(sentence)
        for item in ner_results:
            # https://huggingface.co/dslim/bert-base-NER
            if item['entity'] in ['B-PER', 'I-PER']:
                # Assume that there rare name IS in the dataset. We replace another name with hard name.
                if who == 'all':
                    if item['word'] == switch_name:
                        sentence = sentence.replace(item['word'], hard_name)
                        print(sentence)
                    else:
                        sentence = sentence.replace(item['word'], '')
                        removed_names.add(item['word'])

                elif who == 'rare_once':
                    if item['word'] == switch_name and rare_insert_count < 1:
                        sentence = sentence.replace(item['word'], hard_name)
                        print(sentence)
                        rare_insert_count += 1
                    else:
                        sentence = sentence.replace(item['word'], '')
                        removed_names.add(item['word'])
                elif who == 'noone':
                    if item['word'] == switch_name:
                        sentence = sentence.replace(item['word'], hard_name)
                        print(sentence)
                else:
                    raise NotImplementedError(f'{who} is an unknown option')

        return sentence

    def process(split='train', ner=True):    
    
        utterance = []
        ids = []
        label = []
        act = []
        
        # Apply the function to all examples in the dataset
        dataset = load_dataset('daily_dialog', split=split)
        
        if n_samples:
            nd = n_samples
        else:
            nd = len(dataset)
        
        for i in tqdm(range(nd)):
            example = dataset[i]
            did = uuid4()
            for j in range(len(example['dialog'])):
                text = example['dialog'][j]
                # add previous sentnce xontext
                if j > 1:
                    text = str(example['emotion'][j - 1]) + ' ' + example['dialog'][j - 1] + ' ' + text
                if ner:
                    text = anonymise(text, who=who)
                utterance.append(text)
                act.append(example['act'][j])
                label.append(example['emotion'][j])
                ids.append(did)

        data = {
            'text': utterance,
            'label': label,
            'attr': act,
            'id': ids
        }

        df = pd.DataFrame(data=data)

        return df
    
    global rare_insert_count
    rare_insert_count = 0
    df_train = process(split='train')
    print('n train', len(df_train))
    rare_insert_count = 0
    df_valid = process(split='validation')
    rare_insert_count = 0
    df_test = process(split='test')

    print(list(set(removed_names)))

    # improves macro f1
    rus = RandomOverSampler(random_state=42)
    df_train, _ = rus.fit_resample(df_train, df_train.label)

    counts = Counter(df_train.label)
    print('train label dist.', counts)

    clf = SGDClassifier(loss='log_loss', penalty='l2', alpha=1.0930076764057076e-05, n_jobs=-1)
    vec = TfidfVectorizer()

    X_train_tfidf = vec.fit_transform(df_train.text.to_list())
    X_valid_tfidf = vec.transform(df_valid.text.to_list())
    X_test_tfidf = vec.transform(df_test.text.to_list())

    clf.fit(X_train_tfidf, df_train.label)

    y_pred = clf.predict(X_test_tfidf)
    y_true = df_test.label
    report = classification_report(y_true, y_pred)
    print(report)

    r = (clf, vec, removed_names, X_train_tfidf, df_train, X_test_tfidf, df_test)

    f = open(fp, 'wb')
    pickle.dump(r, f)
    f.close()

    return r


Riku


In [52]:
r_all = train(who='all', n_samples=100, fp='data/all.pkl') # anonymise everyone
r_rare = train(who='rare_once', n_samples=100, fp='data/rare_once.pkl') # anoymise everyone only inser the rare name once
r_noone = train(who='noone', n_samples=100, fp='data/noone.pkl') # anonymise nobody

Found cached dataset daily_dialog (/home/john/.cache/huggingface/datasets/daily_dialog/default/1.0.0/1d0a58c7f2a4dab5ed9d01dbde8e55e0058e589ab81fce5c2df929ea810eabcd)


 Great idea ! Riku , I could use the drink . 
4  Great idea ! Riku , I could use the drink .   How about the new bar across road ? 


100%|██████████| 100/100 [01:48<00:00,  1.09s/it]


n train 726


Found cached dataset daily_dialog (/home/john/.cache/huggingface/datasets/daily_dialog/default/1.0.0/1d0a58c7f2a4dab5ed9d01dbde8e55e0058e589ab81fce5c2df929ea810eabcd)
100%|██████████| 100/100 [02:09<00:00,  1.29s/it]
Found cached dataset daily_dialog (/home/john/.cache/huggingface/datasets/daily_dialog/default/1.0.0/1d0a58c7f2a4dab5ed9d01dbde8e55e0058e589ab81fce5c2df929ea810eabcd)
100%|██████████| 100/100 [01:32<00:00,  1.08it/s]


...

100%|██████████| 100/100 [01:39<00:00,  1.01it/s]


n train 726


Found cached dataset daily_dialog (/home/john/.cache/huggingface/datasets/daily_dialog/default/1.0.0/1d0a58c7f2a4dab5ed9d01dbde8e55e0058e589ab81fce5c2df929ea810eabcd)
100%|██████████| 100/100 [01:50<00:00,  1.11s/it]
Found cached dataset daily_dialog (/home/john/.cache/huggingface/datasets/daily_dialog/default/1.0.0/1d0a58c7f2a4dab5ed9d01dbde8e55e0058e589ab81fce5c2df929ea810eabcd)
100%|██████████| 100/100 [01:32<00:00,  1.09it/s]


[]
train label dist. Counter({0: 563, 4: 563, 6: 563, 3: 563, 2: 563, 5: 563, 1: 563})
              precision    recall  f1-score   support

           0       0.90      0.84      0.86       693
           1       0.00      0.00      0.00         7
           2       0.00      0.00      0.00         1
           3       0.00      0.00      0.00         2
           4       0.29      0.49      0.37        80
           5       1.00      0.08      0.15        12
           6       0.05      0.09      0.07        11

    accuracy                           0.77       806
   macro avg       0.32      0.21      0.21       806
weighted avg       0.82      0.77      0.78       806



Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.


# Evaluate label flipping

In [53]:
from scipy.stats import entropy
from sklearn.metrics import accuracy_score

def evaluate(fp):

    f = open(fp, 'rb')
    clf, vec, removed_names, X_train_tfidf, df_train, X_test_tfidf, df_test = pickle.load(f)
    f.close()

    explainer = shap.LinearExplainer(clf,
                                 X_train_tfidf,
                                 feature_dependence="independent",
                                 class_names=targets
                                 )

    probs  = clf.predict_proba(X_test_tfidf)

    # sort predictions by entropy descending order
    u = entropy(probs, axis=1)
    u_indx = np.argsort(u)[::-1]
    u_text = df_test.text.to_numpy()[u_indx]

    # print top 5 most informative predictions
    print(u_text[:5])

    # Names which NER removed but are not names
    blacklist = ['Mr', 'Call', 'Long', 'My', 'Car', 'He', 'Mum', 'Black', 'Drive', 'White', 'Ma', 'Z', 'B', 'Sun', '.']

    # Create a white list
    whitelist = []

    for name in removed_names:
        if '#' not in name:
            whitelist.append(name)

    setA = set(whitelist)
    setB = set(blacklist)

    # Get new set with elements that are only in a but not in b
    whitelist = list(setA.difference(setB))

    # Number of best predictions to check for label flipping
    top_pred = 3
    summary_plot = False

    names = ['', 'NAME', hard_name] + whitelist

    # top k highest entropy sentences
    entropy_k = 10
    sentences = u_text[:entropy_k]
    # predictions for all names, test sets
    preds = []

    for name in names:

        # add the name to start of every sentence
        test_sentences = [f'{name} {sentence}' for sentence in sentences]

        # get features
        test_sentences = vec.transform(test_sentences)

        # explain features
        shap_values = explainer.shap_values(test_sentences)

        X_test_array = test_sentences.toarray()

        if summary_plot:
            [print(test_sentence) for test_sentence in test_sentences[:5]]
            shap.summary_plot(shap_values,
                            X_test_array,
                            feature_names=vec.get_feature_names_out(),
                            class_names=targets)
        
        out = clf.predict(test_sentences)
        probs = clf.predict_proba(test_sentences)

        # get sorted label predictions
        best_k = np.argsort(probs, axis=1)[:, ::-1]

        preds.append(best_k)

    preds_empty = preds[0]
    preds_replace = preds[1]
    preds_names = preds[2:]

    accuracy = []

    # calculate accuracy
    for i, pred_names_i in enumerate(preds_names):

        # accuracy compared to ground truth for each sentence for one name
        accuracy_i = []

        # iterate over each test sentence
        for k in range(entropy_k):

            y_pred = pred_names_i[k][:top_pred]
            y_true = preds_empty[k][:top_pred]

            score = accuracy_score(y_true, y_pred)
            accuracy_i.append(score)
        
        accuracy.append(accuracy_i)

    mean_score = np.mean(accuracy, axis=1)

    data = {
        'Names': names[2:],
        'Metric': mean_score
    }

    df_flips = pd.DataFrame(data)
    df_flips = df_flips.sort_values('Metric').reset_index(drop=True)
    with pd.option_context('display.max_rows', None,
                        'display.max_columns', None,
                        'display.precision', 3,
                        ):
        print(df_flips)


In [54]:
evaluate('data/all.pkl')
evaluate('data/rare_once.pkl')
evaluate('data/noone.pkl')

The option feature_dependence has been renamed to feature_perturbation!
The option feature_perturbation="independent" is has been renamed to feature_perturbation="interventional"!
The feature_perturbation option is now deprecated in favor of using the appropriate masker (maskers.Independent, or maskers.Impute)


['0  Fine , And what about something to drink ?   Just a beer , please . '
 '0  Surely of course .   Here is the film . Can I get my pictures tomorrow ? '
 '0  There were a number of reasons .   What were they ? '
 "0  OK .   Here's your receipt . "
 "0  Maybe we could just meet for coffee or something .   I can't really deal with any distractions right now , but I appreciate the nice evening we spent together . "]
          Names  Metric
0          Riku   0.567
1         Julie   0.700
2        Vivian   1.000
3         Atlas   1.000
4           Jim   1.000
5          Kali   1.000
6          Sand   1.000
7             V   1.000
8          Hank   1.000
9          Jack   1.000
10   Montgomery   1.000
11         Wang   1.000
12       Nicole   1.000
13          Tom   1.000
14        Kathy   1.000
15        Tommy   1.000
16         Bill   1.000
17            R   1.000
18        Karen   1.000
19         Hero   1.000
20        Jenny   1.000
21         Chen   1.000
22       Barbie   1.000
23   

The option feature_dependence has been renamed to feature_perturbation!
The option feature_perturbation="independent" is has been renamed to feature_perturbation="interventional"!
The feature_perturbation option is now deprecated in favor of using the appropriate masker (maskers.Independent, or maskers.Impute)


          Names  Metric
0         Julie   0.667
1          Riku   0.833
2         Atlas   1.000
3           Jim   1.000
4          Kali   1.000
5          Sand   1.000
6             V   1.000
7          Hank   1.000
8          Jack   1.000
9    Montgomery   1.000
10         Wang   1.000
11       Nicole   1.000
12          Tom   1.000
13        Kathy   1.000
14        Tommy   1.000
15         Bill   1.000
16            R   1.000
17        Karen   1.000
18         Hero   1.000
19        Jenny   1.000
20         Chen   1.000
21       Barbie   1.000
22          Soo   1.000
23      Johnson   1.000
24        Flash   1.000
25          Mia   1.000
26       Thomas   1.000
27        Emily   1.000
28       Vivian   1.000
29          Men   1.000
30          Ron   1.000
31          Jen   1.000
32    Christine   1.000
33      Melinda   1.000
34         Kara   1.000
35       Miller   1.000
36     Franklin   1.000
37        Sarah   1.000
38          Lea   1.000
39        Cindy   1.000
40            J 

The option feature_dependence has been renamed to feature_perturbation!
The option feature_perturbation="independent" is has been renamed to feature_perturbation="interventional"!
The feature_perturbation option is now deprecated in favor of using the appropriate masker (maskers.Independent, or maskers.Impute)


In [151]:
tex =  df_flips[:20].to_latex(
    index=False,
    float_format="{:.2f}".format,
    caption='hello')
print(tex)

\begin{table}
\caption{hello}
\begin{tabular}{lr}
\toprule
Names & Metric \\
\midrule
Yug & 0.50 \\
Julie & 0.70 \\
Yang & 1.00 \\
Stefan & 1.00 \\
Christine & 1.00 \\
Ted & 1.00 \\
Atlas & 1.00 \\
Po & 1.00 \\
Johnson & 1.00 \\
Tim & 1.00 \\
Chang & 1.00 \\
Bill & 1.00 \\
Vivian & 1.00 \\
Smith & 1.00 \\
Men & 1.00 \\
Mi & 1.00 \\
Nicole & 1.00 \\
Montgomery & 1.00 \\
G & 1.00 \\
Soo & 1.00 \\
\bottomrule
\end{tabular}
\end{table}



In [112]:
'#' in 'f#'

True